In [1]:
import tensorflow as tf
import numpy as np


C:\Users\zhaon\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#get cifar10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data();
print(x_train.shape)
print(y_train.shape)


(50000, 32, 32, 3)
(50000, 1)


In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale
onehot_encoder = OneHotEncoder(sparse=False)
y_train_ohe = onehot_encoder.fit_transform(y_train);
print(y_train_ohe.shape)
print(type(y_train_ohe))
y_test_ohe =onehot_encoder.fit_transform(y_test); 


(50000, 10)
<class 'numpy.ndarray'>


In [9]:
#take a batch
batch_size = 1000
[N,H,W,C] = x_train.shape;
num_batches = int(N/batch_size);


# define a resnet architecture using layers api
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3], name = 'X')
input_labels = tf.placeholder(tf.int32, shape = [None, 10], name = 'y')
inputs = X;
for i in range(20):
    filters = 3; kernel_size = (3,3);
    z1 = tf.layers.conv2d(inputs,filters, kernel_size, padding = 'same');
    #z2 = tf.layers.max_pooling2d(z1, (2,2), strides = (1,1))
    z3 = tf.layers.batch_normalization(z1); 
    #input goes before relu
    if(i%2 == 0):
        z4 = tf.nn.relu(z3+inputs);
    else:
        z4 = tf.nn.relu(z3);

    ## to make this a resnet, just add in the inputs here (f(x)+x); 
    # the challenge is that the input has been conved and maxpooled..
    #doesn't seem like resnetting the conv layers works
    inputs = z4;

        

flat = tf.layers.flatten(z4);

d1 = tf.layers.dense(flat, 100, activation = tf.nn.relu);

scores = tf.layers.dense(d1, 10);

loss= tf.nn.softmax_cross_entropy_with_logits_v2(labels = input_labels, logits = scores);
loss = tf.reduce_mean(loss);
optimizer = tf.train.AdamOptimizer();
optimizer = optimizer.minimize(loss);

init = tf.global_variables_initializer()
sess = tf.Session();
sess.run(init)
epochs = 64; print(num_batches)
for i in range(epochs):
    for j in range(num_batches):
        X_batch = x_train[j*batch_size:(j+1)*batch_size, :, :, :];
        y_batch = y_train_ohe[j*batch_size:(j+1)*batch_size, :];
        y_batch_flat = y_train[j*batch_size:(j+1)*batch_size, :]        
        sess.run(optimizer, feed_dict = {X: X_batch, input_labels: y_batch})
        #if(j == num_batches-1): print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))


    print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))
        


50
1.918337
1.7777871
1.672635
1.6037912
1.5446895
1.514097
1.4640743
1.4218432
1.4264026
1.4171355
1.4152231
1.3823775
1.3651155
1.3706701
1.2948895
1.3256818
1.2790327
1.2206236
1.305268
1.2611796
1.1988424
1.1348256
1.1241255
1.134495
1.1326243
1.085825
1.0864581
1.0440274
1.0822233
1.1361177
1.105918
1.133069
1.1186292
1.0946678
1.1226441
1.1424261
1.2278241
1.1803416
1.1117295
1.0633464
1.0485542
1.0295331
0.9982431
1.0712858
1.0617619
1.1447258
1.1032027
1.0511764
1.0252894
1.0184307
1.0945373
1.1329823
1.0591999
0.9658932
0.972596
0.9744271
1.0105498
0.96760476
0.8955789
1.0482345
0.9974572
0.97386086
0.98571116
0.98269475


In [10]:
test_score = 0;
for j in range(int(10000/100)):
    start = j*100; end = (j+1)*100
    test_scores = sess.run(scores, feed_dict = {X: x_test[start:end, :,:,:],\
                                                input_labels: y_test_ohe[start:end, :]});
    test_labels = np.argmax(test_scores, axis = 1);
    test_score+=((1/100)*np.sum(test_labels == np.reshape(y_test[start:end], (100, ))))
    
print(test_score/int(10000/100))

0.3868000000000002


In [8]:
#training loss
print(sess.run(loss, feed_dict = {X: x_test, input_labels: y_test_ohe}))
print(y_batch.shape)
#get test accuracy;
test_scores = sess.run(scores, feed_dict = {X: x_train, input_labels: y_train_ohe});
test_labels = np.argmax(test_scores, axis = 1);
print((1/50000)*np.sum(test_labels == np.reshape(y_train, (50000, ))))

print(y_batch_flat.shape)
print(test_labels.shape)

3.36208
(1000, 10)
0.65322
(1000, 1)
(50000,)


In [19]:
''' 
train a resnet with flexible resizing
in this situation, ALL filters are the same size, but greater than 3 so the only input
that needs padding is the X
'''

#take a batch
batch_size = 1000
[N,H,W,C] = x_train.shape;
num_batches = int(N/batch_size);


# define a resnet architecture using layers api
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3], name = 'X')
X_padding = tf.pad(X, [[0,0],[0,0],[0,0],[14, 15]])
print(X_padding.shape)
input_labels = tf.placeholder(tf.int32, shape = [None, 10], name = 'y')
inputs = X;
for i in range(20):
    filters = 32; kernel_size = (3,3);
    z1 = tf.layers.conv2d(inputs,filters, kernel_size, padding = 'same');
    #z2 = tf.layers.max_pooling2d(z1, (2,2), strides = (1,1))
    z3 = tf.layers.batch_normalization(z1); 
    #input goes before relu
    if(i%2 == 0):
        #pad the inputs so they are the same size as z3
        if(i == 0):
            padded_inputs = X_padding
        else:
            padded_inputs = inputs;
        z4 = tf.nn.relu(z3+padded_inputs);
    else:
        z4 = tf.nn.relu(z3);

    ## to make this a resnet, just add in the inputs here (f(x)+x); 
    # the challenge is that the input has been conved and maxpooled..
    #doesn't seem like resnetting the conv layers works
    inputs = z4;

        

flat = tf.layers.flatten(z4);

d1 = tf.layers.dense(flat, 100, activation = tf.nn.relu);

scores = tf.layers.dense(d1, 10);

loss= tf.nn.softmax_cross_entropy_with_logits_v2(labels = input_labels, logits = scores);
loss = tf.reduce_mean(loss);
optimizer = tf.train.AdamOptimizer();
optimizer = optimizer.minimize(loss);

init = tf.global_variables_initializer()
sess = tf.Session();
sess.run(init)
epochs = 64; print(num_batches)
for i in range(epochs):
    for j in range(num_batches):
        X_batch = x_train[j*batch_size:(j+1)*batch_size, :, :, :];
        y_batch = y_train_ohe[j*batch_size:(j+1)*batch_size, :];
        y_batch_flat = y_train[j*batch_size:(j+1)*batch_size, :]        
        sess.run(optimizer, feed_dict = {X: X_batch, input_labels: y_batch})
        #if(j == num_batches-1): print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))


    print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))
        

(?, 32, 32, 32)
50
1.9628592
1.6415296
1.483886
1.3569304
1.30216
1.2294301
1.1243703
1.0505928
1.0054898
0.95571196
0.8728382
0.85331106
0.8252587
0.76228166
0.67413783
0.6818821
0.6991049
0.6624811
0.6561026
0.5812231
0.51621795
0.43775374
0.4962365
0.44061223
0.3394354
0.39229926
0.4982153
0.44162065
0.35921803
0.34465733
0.35259286
0.4928921
0.39488354
0.30986664
0.28232145
0.32334545
0.33965355
0.30018803
0.34975445
0.29946703
0.30915052
0.29006675
0.20330288
0.2730444
0.2215036
0.23251429
0.2499972
0.20044991
0.18913446
0.15270746
0.1777589
0.1374425
0.14991327
0.122311786
0.08652727
0.062134016
0.08820142
0.042586353
0.06600661
0.048490852
0.07565797
0.07529184
0.042257413
0.044459343


In [20]:
test_score = 0;
for j in range(int(10000/100)):
    start = j*100; end = (j+1)*100
    test_scores = sess.run(scores, feed_dict = {X: x_test[start:end, :,:,:],\
                                                input_labels: y_test_ohe[start:end, :]});
    test_labels = np.argmax(test_scores, axis = 1);
    test_score+=((1/100)*np.sum(test_labels == np.reshape(y_test[start:end], (100, ))))
    
print(test_score/int(10000/100))

0.5475999999999999
